# 외부와 상호작용 하면서 자율적으로 행동하는 Agents
## 언어 모델에 도구를 부여할 수 있다.
- Agents 모듈을 사용해 다양한 작업에 맞는 도구를 선택해 실행할 수 있다.
- 다음과 같은 두 가지 하위 모듈이 있다.
    - Tool
    - Agent
### 말 그대로 다양한 **도구**가 되는 Tool
- Tool은 언어 모델만으로 할 수 없는 일을 할 수 있게 해주는 모듈
- 계산, 검색 등 다양한 종류가 있어 목적에 따라 선택
    - LLMMath
        - 언어 모델이 취약한 계산
    - Requests
        - 지정된 URL로 요청 보낼 수 있게 해줌, 웹사이트의 정보 취득, 인터넷에 공개된 API 사용 등
    - File System Tools
        - PC내 파일에 접근, 지정된 경로에서 파일 읽고 쓰기 가능, 실행 결과 저장 등
    - SerpApi
        - 구글이나 야후 검색을 API로 하는 SerpApi 라는 웹 서비스와 연동동

### 명령에 따라 자율적으로 처리를 수행하는 Agent
- Agent는 Tool을 선택 하고 다음 단계의 처리를 수행하는 주체
- ReAct 기법 
    1. 사용자로 부터 작업을 받는다
    2. 준비된 Tool 중에서 어떤 Tool을 사용할지, 어떤 정보를 입력 할 지 결정
    3. Tool를 사용해 결과를 얻는다
    4. 3에서 얻은 결과를 통해 과업이 달성되고 있는지 확인한다
    5. 에이전트가 작업을 완료했다고 판단할 수 있을 때까지 2~4를 반복한다 

## 주어진 URL에서 정보 얻게 하기
- URL에 엑세스 할 수 있는 Tool을 가진 Agent를 만들어 보자

In [16]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI

# 모델 초기화
chat = ChatOpenAI(
    temperature=0, # 응답 다양성 설정, 0~2, 기본값은 0.7
    model='gpt-3.5-turbo'
)

tools = load_tools( # 렝체인에 준비된 Tool 로드
    [
        "requests_get", # 특정 URL의 결과를 얻는 Tool, requests과 requests_get는 보안상 문제가 있음
    ],
    allow_dangerous_tools=True # 이 옵션을 주어야만 사용 가능
)

agent = initialize_agent( # Agent 초기화
    tools=tools, # 사용할 tools 리스트
    llm=chat, # 언어모델 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, # ReAct 방식으로 작동하게 설정
    verbose=True # 로그 표시
)

In [17]:
result = agent.run("""아래 URL에 접속해 도쿄의 날씨를 검색해 한국어로 말하세요.
https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json
""")



> Entering new AgentExecutor chain...
Question: Access the URL below and search for the weather in Tokyo. Please say it in Korean.
https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json
Thought: I should use the requests_get tool to retrieve the weather information for Tokyo from the provided URL.
Action:
```
{
  "action": "requests_get",
  "action_input": "https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json"
}
```

Observation: {"publishingOffice":"気象庁","reportDatetime":"2024-05-27T04:35:00+09:00","targetArea":"東京都","headlineText":"","text":"　日本海には低気圧があって、前線が朝鮮半島付近にのびています。また、日本のはるか東には高気圧があって、関東甲信地方は湿った空気の影響を受けています。\n\n　東京地方は、曇りとなっています。\n\n　２７日は、日本海を前線が東進し、湿った空気の影響を受ける見込みです。このため、曇りで昼過ぎは雨となるでしょう。伊豆諸島では雨で雷を伴う所がある見込みです。\n\n　２８日は、朝には九州付近に低気圧が発生し、夜にかけて東日本に進む見込みです。このため、曇り朝晩雨で、夜遅くは雷を伴い激しい雨の降る所があるでしょう。伊豆諸島では雨で雷を伴い激しく降る所がある見込みです。\n\n【関東甲信地方】\n　関東甲信地方は、曇りで雨の降っている所があります。\n\n　２７日は、日本海を前線が東進し、湿った空気の影響を受ける見込みです。このため、曇りで、昼前から昼過ぎを中心に雨が降るでしょう。伊豆諸島では夜は雷雨となる所がある見込みで

In [19]:
print(f'실행 결과 : {result}')

실행 결과 : 도쿄 지역은 오늘 날씨가 흐린 상태이며, 오후에 비가 올 것으로 예상됩니다.


#### Agent의 종류
- Agent 는 Tool을 사용 특정 테스크를 달성 한다. Agent를 조작하는 방법은 다양하며 이에 따라 툴의 조작 방법도 다르다.
    - [link](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html)
    - AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION
        - 대화 형식의 상호 작용이 특징, Chat models 모듈과 ReAct 방법을 사용해 작동
    - AgentType.ZERO_SHOT_REACT_DESCRIPTION
        - LLMs 모듈 사용을 전제로 하는 에이전트
    - AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
        - 복수개의 inputs 을 가진 툴을 사용

# Tool을 추가해 Agent가 할 수 있는 일 늘리기

## Agent가 할 수 있는 것은 전달하는 Tool에 따라 달라진다
- 파일 스기와 인터넷 검색을 할 수 있는 Tool을 추가 해보자
- SerpApi 라는 서비스를 사용 인터넷에서 정보를 가져 온다.
    - 구글 등 검색 엔진의 검색 결과를 가져올 수 있는 서비스

## 환경 변수에 SerpApi의 API Key 설정하기
- 각 OS에 맞게 API를 환경 변수에 등록해 준다.

## google-search-results 설치하기
- SerpApi에서 구글 검색을 하려면 google-search-results 라는 파이썬 페키지 필요
- pip install google-search-results 

In [9]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.tools.file_management import WriteFileTool # 파일 쓰기를 할 수 있는 Tool

chat = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

tools = load_tools(
    [
        'requests_get',
        'serpapi'
    ],
    allow_dangerous_tools =True,
    llm=chat
)

tools.append(WriteFileTool(
    root_dir="./"
))

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # WriteFIleTool과 같이 복수개의 입력을 받을 수 있는 Tool 사용 가능,
    verbose=True
)

In [10]:
result = agent.run("경주시의 특산품을 검색해 result.txt 파일에 한국어로 저장하세요.")



> Entering new AgentExecutor chain...
이 작업을 수행하기 위해 경주시의 특산품을 검색하여 결과를 파일에 저장해야 합니다. 검색 결과를 얻기 위해 Search 도구를 사용하고, 결과를 파일에 저장하기 위해 write_file 도구를 활용할 것입니다. 시작해 보겠습니다. 

Action:
```
{
  "action": "Search",
  "action_input": "경주시 특산품"
}
```
Observation: ['감포특산물 type: 상점.', '감포특산물 entity_type: local_nav.', '감포특산물 kgmid: /g/11f64dh3rn.', '감포특산물 place_id: ChIJ6yZGuJalZzURa8N7WR5ie0A.', '감포특산물 phone: +82 54-774-6889.', '감포특산물 address: South Korea, Gyeongsangbuk-do, Gyeongju-si, Gampo-eup, Gampo-ri, 484-3 감포읍 감포리 484-3번지 경주시 경상북도 KR 38125.', '감포특산물 raw_hours: Open ⋅ Closes 8 PMMonday10 AM–8 PMTuesday10 AM–8 PMWednesday10 AM–8 PMThursday10 AM–8 PMFriday10 AM–8 PMSaturday10 AM–8 PMSunday10 AM–8 PMSuggest new hoursUnable to add this file. Please check that it is a valid photo..', '경주시 대표 과채류로 형상강 하류의 유기물이 풍부하고 물빠짐이 좋은 토양환경과 평야지에서 많은 일조량을 받고 자라 과육이 풍부합니다. 당도가 높은 동양계 품종으로 ...', '교동법주 정보 · 전화 054-772-2051 · 주소 교촌안길 19-21 · 가격 교동법주900ml(종이상자) 35,000원 / 교동법주900ml(나무상자) 40,000원 ...', '어느지역마다 그 지역을 대

### ReAct 기법과 OpenAI Function Calling
- OpenAI의 언어모델만 사용한다면,
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION 대신에 OPENAI_FUNCTIONS (단일 입력)
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 대신에 OPENAI_MULTI_FUNCTIONS (복수 입력)
- 을 사용해도 유사한 결과가 나온다.

In [11]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.tools.file_management import WriteFileTool # 파일 쓰기를 할 수 있는 Tool

chat = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

tools = load_tools(
    [
        'requests_get',
        'serpapi'
    ],
    allow_dangerous_tools =True,
    llm=chat
)

tools.append(WriteFileTool(
    root_dir="./"
))

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS, # WriteFIleTool과 같이 복수개의 입력을 받을 수 있는 OpenAI의 Function callings
    verbose=True
)

In [12]:
result = agent.run("경주시의 특산품을 검색해 result.txt 파일에 한국어로 저장하세요.")



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'tool_input': 'Specialty products of Gyeongju'}`


['Bulgogi (broiled meat) · Hwangnam-ppang (Gyeongju bread) · Garlic chicken · Chueo-tang (loach soup) · Hoe (sashimi) · Sundubu-jjigae (soft tofu stew) · ...', 'Gift & Specialty Shops in Gyeongju · Good for Kids · Good for Big Groups · Free Entry · Adventurous · Budget-friendly · Good for a Rainy Day · Hidden Gems · Good for ...', 'Gyeongju was the capital of the ancient kingdom of Silla (57 BC – 935 AD), which ruled about two-thirds of the Korean peninsula at its height between the 7th ...', 'Travel back to the Silla era with the items defining the Gyeongju National Museum · 1. Gold Crown · 2. Divine Bell of King Seongdeok · 3. Bhaisajyaguru Buddha · 4.', "Hwangnam bread is one of the many specialties you'll find in Gyeongju. This artisan bread is a red bean pastry that was first baked by Young-hwa Choi in ...", 'Sample the specialties of Gyeongju cuisine. Gyeongju is 

# Tool을 직접 제작해 기능 확장하기
## Tool을 직접 만들어서 할 수 있는 일의 폭을 더욱 넓힌다
- Agent 모듈에서는 사용자가 직접 Tool을 쉽게 생성 가능하다.
- Tool의 구성 요소와 Agent가 Tools 선택 하는 메커니즘을 우선 살펴 보자
- Tool은 다음과 같은 세 가시 요소로 구성되어 있다.
    - 이름 : Tool을 식별하기 위한 이름 예) Calculator, requests_get
    - 기능 설명 : Tool이 무엇을 하는지에 대한 간략한 설명, Agent가 Tool을 어떻게 사용할지 판단 할 수 있는 자료
    - 실행 함수 : Tool이 실제 작동할 때 처리

In [17]:
from langchain_community.llms import Ollama
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent


chat = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

df = pd.read_csv('/home/restful3/datasets_bigpond/kaggle/titanic/train.csv', low_memory=False)
agent = create_pandas_dataframe_agent(chat, df, verbose=True)

query = "how many rows are there?"
agent.run(query)



> Entering new AgentExecutor chain...
Thought: I can use the shape attribute of the dataframe to determine the number of rows.
Action: python_repl_ast
Action Input: df.shape[0]891I now know the final answer
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'